The purpose of this program is to analyse the basic properties of rational functions with one variable. The resulting table contains the roots, local extrema, discontinuity points, and inflection points of the function, along with the monotonicity of the function between these critical points.

The behaviour line in the table denotes the monotonicity of the function between the points, and the function value at the critical points.

In [3]:
from sympy import Symbol, Poly, diff, simplify, fraction
from tabulate import tabulate
import sympy as sym
import numpy as np

def signof(a):
    if a>0:
        return "+"
    if a==0:
        return 0
    else:
        return "-"

def analyze_function(num, denom, x, digits=3):
    
    def calcroots(coeffs):
        r = np.roots(coeffs)
        approx = [round(val,2*digits) for val in r.real[abs(r.imag)<1e-5]]
        return approx
    
    numerator = Poly(num, x)
    denominator = Poly(denom, x)
    num_coeffs = numerator.all_coeffs()
    f = num/denom
    
    derivative = simplify(diff(f, x))
    second_derivative = simplify(diff(derivative, x))
    third_derivative = simplify(diff(second_derivative, x))
    
    print("The function: ", f)
    derivative_coeffs = Poly(fraction(derivative)[0], x).all_coeffs()
    second_derivative_coeffs = Poly(fraction(second_derivative)[0], x).all_coeffs()

    roots = calcroots(num_coeffs)
    extrema = calcroots(derivative_coeffs)
    infl_points = calcroots(second_derivative_coeffs)
    discontinuities = [sing.evalf(2*digits) for sing in sym.singularities(f,x)]

    critical_points = sorted(list(set(roots + extrema + infl_points + discontinuities)))

    def getconcavity(p):
        conc = ""
        if signof(second_derivative.subs(x,p)) == "+":
            conc += "convex"
        else:
            conc += "concave"
        return conc

    def gettype(p):
        typ = ""
        if p in roots:
            typ += "root, "
        if p in extrema:
            if signof(second_derivative.subs(x,p)) == "+":
                typ += "min, "
            elif signof(second_derivative.subs(x,p)) == "-":
                typ += "max, "
        if p in infl_points:
            typ += "infl, "
        if p in discontinuities:
            typ += "disc, "
        return typ[:-2]

    first_row = ["Critical points: "]
    second_row = ["Properties: "]
    third_row = ["Behaviour: "]
    for i in range(len(critical_points)):
        point = critical_points[i]
        try:
            next_point = critical_points[i+1]
        except:
            next_point = point + 1
        if i==0:
            first_row.append("")
            second_row.append(getconcavity(point-1))
            third_row.append(signof(derivative.subs(x,point-1)))
        if round(round(point,digits)) == round(point,digits):
            first_row.append(round(point))
        else:
            first_row.append(round(point,digits))
        first_row.append("")
        second_row.append(gettype(point))
        second_row.append(getconcavity((point+next_point)/2))
        if "disc" in gettype(point):
            third_row.append(sym.oo)
        elif round(round(f.subs(x,point),digits)) == round(f.subs(x,point),digits):
            third_row.append(round(f.subs(x,point)))
        else:
            third_row.append(round(f.subs(x,point),digits))
        third_row.append(signof(derivative.subs(x,(point + next_point)/2)))

    table = [first_row, second_row, third_row]
    
    return tabulate(table)
    
    
    
x = Symbol("x", real=True)

print(analyze_function(x**2+x**5, 2+2*x+x**2, x))

print(analyze_function(x**3-3*x**2+2, 1, x))

print(analyze_function(1, x, x))

print(analyze_function(x**3, 1, x))



The function:  (x**5 + x**2)/(x**2 + 2*x + 2)
----------------  -------  ----  -------  -----  -------  ------  ------  ---------  ------
Critical points:           -1.0           -0.76           -0.527          0.0
Properties:       concave  root  concave  max    concave  infl    convex  root, min  convex
Behaviour:        +        0     +        0.306  -        0.194   -       0          +
----------------  -------  ----  -------  -----  -------  ------  ------  ---------  ------
The function:  x**3 - 3*x**2 + 2
----------------  -------  ------  -------  ---  -------  ----------  ------  ---  ------  -----  ------
Critical points:           -0.732           0.0           1.0                 2.0          2.732
Properties:       concave  root    concave  max  concave  root, infl  convex  min  convex  root   convex
Behaviour:        +        0       +        2    -        0           -       -2   +       0      +
----------------  -------  ------  -------  ---  -------  ----------  ---